In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import pickle, random, datetime, numpy as np

from environments.env import Environment
# from environments.make_envs import make_sensor_based_mdp, HEIGHT
# from environments.episodic_mdp import EpisodicMDP

# from agent.agents import NoisyDriverAgent, OptimalAgent, CopyAgent

# from experiments.pre_train_machine_policy import run_pre_train_machine
# from experiments.utils import get_trajectory_cost
# from experiments.single_team import run_single_team_exp
# from experiments.multiple_teams import run_multiple_teams_exp


# from plot.plot_regret import format_axes, latexify, COLORS, golden_ratio
from plot.plot_path import PlotPath

# np.random.seed(981276345)
# random.seed(981276345)

## Environment trajectories

In [3]:
''' Figure 1 '''
%matplotlib inline
from agent.agents import NoisyDriverAgent

width, height = 3, 10
def plot_trajectory(init_traffic):
    env = Environment()
    grid_world = env.generate_grid_world(width, height, init_traffic_level=init_traffic)
    plt_path = PlotPath(grid_world, n_try=0)
        
    # plt_path.plot(f'outputs/envs/{init_traffic}_init_traffic_level.png')
    plt_path.plot(f'{init_traffic}_init_traffic_level_test.png')
    
    # fast check if human is working. To be removed 
    human =  NoisyDriverAgent(env,2)
    grid_world.reset()
    curr_cell = grid_world.cell_types[grid_world.current_coord]
    next_r_traffic = grid_world.traffic_levels[1]
    s0 = [curr_cell, next_r_traffic]
    for a in range(3):    
        next_coords,_ = grid_world.next_cell(a, False)
        is_wall = a!=1 and next_coords[0] == grid_world.current_coord[0]  
        next_cell_type= 'wall' if is_wall else grid_world.cell_types[next_coords]
        s0.append(next_cell_type)

    print(s0)
    print(human.take_action(s0))

plot_trajectory('no-car')
plot_trajectory('light')
plot_trajectory('heavy')

['road', 'no-car', 'road', 'road', 'road']
[0]
['road', 'light', 'car', 'road', 'road']
[2]
['road', 'light', 'road', 'road', 'stone']
[1]


## Performance of the pre-trained machine and human policy

In [2]:
''' episodic MDP for the lane driving environment with sensor-based states '''

mdp = EpisodicMDP.load('outputs/MDPs/sensor_MDP')

In [4]:
''' load the pre-trained machine or re-train it '''

with open('outputs/agents/pre_trained_machine/machine_agent', 'rb') as file:
    machine = pickle.load(file)

# to re-train the machine policy, run the following
# machine, _ = run_pre_train_machine(env=Environment(), mdp=mdp, n_episode=5000)

In [5]:
''' human policy '''
human = NoisyDriverAgent(env=Environment(), mdp=mdp, noise_sd=2)
human.update_policy()

In [6]:
''' optimal policy '''

optimal = OptimalAgent(mdp)
optimal.update_policy()

In [7]:
''' performance '''
    
ep_num = 1000
agents = [CopyAgent(mdp, machine.policy), CopyAgent(mdp, human.policy), CopyAgent(mdp, optimal.policy)]
no_car_costs = []
light_heavy_costs = []

# costs for machine, human and optimal policies
for ep in range(ep_num):
    no_car_costs.append(get_trajectory_cost(agents, init_traffic='no-car'))

    traffic_level = random.choice(['light', 'heavy'])
    light_heavy_costs.append(get_trajectory_cost(agents, init_traffic=traffic_level))

print(f'no-car costs:{[sum(x) for x in zip(*no_car_costs)]},\n'
      f'light-heavy costs:{[sum(x) for x in zip(*light_heavy_costs)]}\n')
        
no_car = [sum(x) for x in zip(*no_car_costs)]
light_heavy = [sum(x) for x in zip(*light_heavy_costs)]

no-car costs:[816.0, 2680.0, 756.0],
light-heavy costs:[11842.0, 5528.0, 3292.0]



In [9]:
''' Figure 2, choose `no_car` or `light_heavy` for fig 2(a) and 2(b) respectively'''
array = no_car

width = 1.3
height = 1

latexify(fig_height=height, fig_width=width, font_size=7, legend_size=5)

labels = [r'Machine', r'Human', r'Optimal']
x = np.arange(len(labels)) # label locations

bar_width = 0.5

fig, ax = plt.subplots()
ax = format_axes(ax)
ax.ticklabel_format(style='sci', scilimits=(3, 3), axis='y')
ax.bar(x, array, bar_width, color=[COLORS[2], COLORS[1], COLORS[0]])
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylabel(r'Total environment cost')
ax.set_xlabel(r'Policy')

plt.subplots_adjust(left=0.2, bottom=0.26, right=0.98, top=0.78, wspace=0, hspace=0)
fig.savefig('outputs/agents_performance_no_car.pdf')
plt.close()

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
latex: warning: running with administrator privileges
This is pdfTeX, Version 3.14159265-2.6-1.40.19 (MiKTeX 2.9.6840 64-bit)
entering extended mode
(C:/Users/user/.matplotlib/tex.cache/78f6f39aa5797522ad9ae220d1764233.tex
LaTeX2e <2018-04-01> patch level 5
("C:\Users\user\AppData\Local\Programs\MiKTeX 2.9\tex\latex\base\article.cls"
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
("C:\Users\user\AppData\Local\Programs\MiKTeX 2.9\tex\latex\base\size10.clo"))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.5 \usepackage
               {type1ec}
No pages of output.
Transcript written on 78f6f39aa5797522ad9ae220d1764233.log.




## Single team experiment

In [8]:
""" single team experiment, figure 3 (a), 4 (a) """

# will take ~ 3 hours for 20000 episodes

np.random.seed(12345)
random.seed(12345)

agents = [CopyAgent(mdp, machine.policy), human]
agents_costs = [0, 0]
switching_cost = 0
indices = list(range(10001))
plot_epochs = indices[1:20] + indices[480:501] + indices[2980:3001] + indices[9980:10001]

switching_agents, regrets = (
    run_single_team_exp(env=Environment(), mdp=mdp, n_episode=20001, agents=agents, agents_cost=agents_costs,
                        switching_cost=switching_cost, time_changing=[0], unknown_agents=[1], n_try=1,
                        plot_epochs=plot_epochs, plot_dir='outputs/epoch_plots/without_costs/')
)

2021-02-19 11:43:08.484273, Episode 0,
cumulative costs: {'ucrl2_mc': 0.0, 'ucrl2': 0.0, 'human_only': 2.0, 'machine_only': 0.0}

2021-02-19 11:46:01.506329, Episode 500,
cumulative costs: {'ucrl2_mc': 1428.0, 'ucrl2': 1394.0, 'human_only': 520.0, 'machine_only': 2566.0}

2021-02-19 11:50:08.098815, Episode 1000,
cumulative costs: {'ucrl2_mc': 2436.0, 'ucrl2': 2370.0, 'human_only': 1034.0, 'machine_only': 4752.0}

2021-02-19 11:54:33.677750, Episode 1500,
cumulative costs: {'ucrl2_mc': 3328.0, 'ucrl2': 3174.0, 'human_only': 1800.0, 'machine_only': 7134.0}

2021-02-19 11:59:16.405329, Episode 2000,
cumulative costs: {'ucrl2_mc': 3752.0, 'ucrl2': 3638.0, 'human_only': 2500.0, 'machine_only': 9414.0}

2021-02-19 12:04:55.805930, Episode 2500,
cumulative costs: {'ucrl2_mc': 3878.0, 'ucrl2': 3724.0, 'human_only': 3166.0, 'machine_only': 11694.0}

2021-02-19 12:09:49.011586, Episode 3000,
cumulative costs: {'ucrl2_mc': 4026.0, 'ucrl2': 3864.0, 'human_only': 3904.0, 'machine_only': 14090.0}



In [6]:
""" single team experiment, figure 3 (b), 4 (b) """

# will take ~ 3 hours for 20000 episodes
np.random.seed(12345)
random.seed(12345)

agents = [CopyAgent(mdp, machine.policy), human]
agents_costs = [0.0, 0.2]
switching_cost = 0.1
indices = list(range(10001))
plot_epochs = indices[1:20] + indices[480:501] + indices[2980:3001] + indices[9980:10001]

switching_agents, costs_regrets = (
    run_single_team_exp(env=Environment(), mdp=mdp, n_episode=20001, agents=agents, agents_cost=agents_costs,
                        switching_cost=switching_cost, time_changing=[0], unknown_agents=[1], n_try=1,
                        plot_epochs=plot_epochs, plot_dir='outputs/epoch_plots/with_costs/')
)

2021-02-19 12:16:53.182868, Episode 0,
cumulative costs: {'ucrl2_mc': -0.3999999999999999, 'ucrl2': -0.3999999999999999, 'human_only': 3.4000000000000017, 'machine_only': -0.3999999999999999}

2021-02-19 12:19:24.175166, Episode 500,
cumulative costs: {'ucrl2_mc': 2039.8, 'ucrl2': 2025.7999999999995, 'human_only': 1143.9999999999943, 'machine_only': 2262.199999999999}

2021-02-19 12:22:44.323616, Episode 1000,
cumulative costs: {'ucrl2_mc': 3169.000000000004, 'ucrl2': 2954.9000000000033, 'human_only': 2279.5999999999976, 'machine_only': 4177.7999999999965}

2021-02-19 12:26:32.849410, Episode 1500,
cumulative costs: {'ucrl2_mc': 3986.300000000012, 'ucrl2': 3611.8000000000093, 'human_only': 3597.900000000049, 'machine_only': 6264.099999999996}

2021-02-19 12:30:34.860978, Episode 2000,
cumulative costs: {'ucrl2_mc': 4254.100000000006, 'ucrl2': 3921.7000000000135, 'human_only': 4912.200000000099, 'machine_only': 8218.399999999994}

2021-02-19 12:35:59.802274, Episode 2500,
cumulative cos

In [43]:
''' plot the regret results for single team (Figure 4) '''

# you can run the above cells or load the saved regret results 

# for figure 4(b) load file `switching_regrets_s0`, for figure 4(a) load file `switching_regrets_s01`
with open('outputs/agents/single_team/switching_regrets_s01', 'rb') as file:
    regrets = pickle.load(file)

width = 1.3
height = 1

latexify(fig_height=height, fig_width=width, font_size=7, legend_size=5)
line_width = 0.8
line_style='solid'

plt.figure(1)
fig, ax = plt.subplots()
ax = format_axes(ax)
ax.ticklabel_format(style='sci', scilimits=(3, 3), axis='y')
ax.ticklabel_format(style='sci', scilimits=(3, 3), axis='x')
ax.set_ylabel(r'Regret, $R(T)$')
ax.set_xlabel(r'Episode, $k$')

ax.plot(np.cumsum(regrets['ucrl2_mc']),color=COLORS[0], label=r'UCRL2-MC', linestyle=line_style,
        linewidth=line_width)
ax.plot(np.cumsum(regrets['ucrl2']), color=COLORS[3], label=r'UCRL2', linestyle=line_style,
        linewidth=line_width)
ax.plot(np.cumsum(regrets['machine_only']), color=COLORS[2], label=r'Machine', linestyle=line_style,
        linewidth=line_width)
ax.plot(np.cumsum(regrets['human_only']), color=COLORS[1], label=r'Human', linestyle=line_style,
        linewidth=line_width)

ax.legend(frameon=False, loc=(0.0, 0.4))
plt.subplots_adjust(left=0.23, bottom=0.28, right=0.98, top=0.87, wspace=0, hspace=0)
# plt.show()
fig.savefig('outputs/single_team_s01.pdf')
plt.close()

## Multiple teams experiment

In [5]:
""" multiple team experiment, figure 5 """

# it takes ~ 7 hours for 5000 episodes
np.random.seed(12345)
random.seed(12345)

agents = []
n_teams = 10

for n in range(n_teams):
    noise_sd = random.uniform(0, 4)
    human = NoisyDriverAgent(env=Environment(), mdp=mdp, noise_sd=noise_sd)
    human.update_policy()
    agents.append([CopyAgent(mdp, machine.policy), human])

agents_costs = [0.0, 0.2]
switching_cost = 0.1

_, regrets = (
    run_multiple_teams_exp(env=Environment(), mdp=mdp, n_teams=n_teams, n_episode=5000, agents=agents, agents_cost=agents_costs,
                        switching_cost=switching_cost, time_changing=[0], unknown_agents=[1], n_try=1)
)

2021-02-19 10:55:42.279841, Episode 0, cumulative total regret: {'ucrl2_mc': 33.4, 'ucrl2': 21.4}
2021-02-19 10:55:57.345589, Episode 10, cumulative total regret: {'ucrl2_mc': 532.7, 'ucrl2': 510.70000000000005}
2021-02-19 10:56:14.699808, Episode 20, cumulative total regret: {'ucrl2_mc': 942.1999999999999, 'ucrl2': 956.0}
2021-02-19 10:56:33.681592, Episode 30, cumulative total regret: {'ucrl2_mc': 1458.2, 'ucrl2': 1451.3}
2021-02-19 10:56:54.424789, Episode 40, cumulative total regret: {'ucrl2_mc': 1864.7000000000003, 'ucrl2': 1980.9}
2021-02-19 10:57:17.605681, Episode 50, cumulative total regret: {'ucrl2_mc': 2191.8, 'ucrl2': 2349.4}
2021-02-19 10:57:41.796734, Episode 60, cumulative total regret: {'ucrl2_mc': 2512.9000000000005, 'ucrl2': 2762.5}
2021-02-19 10:58:07.346230, Episode 70, cumulative total regret: {'ucrl2_mc': 2806.200000000001, 'ucrl2': 3159.7999999999997}
2021-02-19 10:58:34.504038, Episode 80, cumulative total regret: {'ucrl2_mc': 3108.500000000001, 'ucrl2': 3577.89

2021-02-19 11:31:08.510413, Episode 680, cumulative total regret: {'ucrl2_mc': 11576.299999999992, 'ucrl2': 26076.0}
2021-02-19 11:31:42.544119, Episode 690, cumulative total regret: {'ucrl2_mc': 11589.799999999994, 'ucrl2': 26321.6}
2021-02-19 11:32:16.010804, Episode 700, cumulative total regret: {'ucrl2_mc': 11675.199999999995, 'ucrl2': 26650.399999999994}
2021-02-19 11:32:51.118353, Episode 710, cumulative total regret: {'ucrl2_mc': 11781.699999999995, 'ucrl2': 26995.099999999995}
2021-02-19 11:33:25.569596, Episode 720, cumulative total regret: {'ucrl2_mc': 11814.299999999996, 'ucrl2': 27295.499999999993}
2021-02-19 11:34:00.256829, Episode 730, cumulative total regret: {'ucrl2_mc': 11876.099999999997, 'ucrl2': 27572.299999999992}
2021-02-19 11:34:35.163934, Episode 740, cumulative total regret: {'ucrl2_mc': 11935.599999999995, 'ucrl2': 27826.899999999987}
2021-02-19 11:35:09.518911, Episode 750, cumulative total regret: {'ucrl2_mc': 12049.499999999993, 'ucrl2': 28164.899999999983

2021-02-19 12:09:44.920911, Episode 1330, cumulative total regret: {'ucrl2_mc': 14811.89999999999, 'ucrl2': 41884.49999999994}
2021-02-19 12:10:21.473596, Episode 1340, cumulative total regret: {'ucrl2_mc': 14866.49999999999, 'ucrl2': 42142.199999999946}
2021-02-19 12:10:58.351300, Episode 1350, cumulative total regret: {'ucrl2_mc': 14900.899999999992, 'ucrl2': 42271.79999999993}
2021-02-19 12:11:34.609448, Episode 1360, cumulative total regret: {'ucrl2_mc': 14984.199999999993, 'ucrl2': 42519.999999999935}
2021-02-19 12:12:11.528747, Episode 1370, cumulative total regret: {'ucrl2_mc': 15011.799999999994, 'ucrl2': 42686.59999999993}
2021-02-19 12:12:48.542300, Episode 1380, cumulative total regret: {'ucrl2_mc': 15043.999999999995, 'ucrl2': 42848.399999999936}
2021-02-19 12:13:25.374027, Episode 1390, cumulative total regret: {'ucrl2_mc': 15036.999999999993, 'ucrl2': 42908.99999999994}
2021-02-19 12:14:02.062348, Episode 1400, cumulative total regret: {'ucrl2_mc': 15053.699999999993, 'uc

2021-02-19 12:49:49.316883, Episode 1980, cumulative total regret: {'ucrl2_mc': 16839.800000000003, 'ucrl2': 49429.09999999997}
2021-02-19 12:50:26.391092, Episode 1990, cumulative total regret: {'ucrl2_mc': 16898.699999999997, 'ucrl2': 49567.39999999997}
2021-02-19 12:51:03.604190, Episode 2000, cumulative total regret: {'ucrl2_mc': 16905.59999999999, 'ucrl2': 49663.89999999997}
2021-02-19 12:51:40.488240, Episode 2010, cumulative total regret: {'ucrl2_mc': 16968.5, 'ucrl2': 49731.89999999998}
2021-02-19 12:52:17.481057, Episode 2020, cumulative total regret: {'ucrl2_mc': 16998.700000000004, 'ucrl2': 49844.99999999998}
2021-02-19 12:52:54.936471, Episode 2030, cumulative total regret: {'ucrl2_mc': 16981.900000000005, 'ucrl2': 49838.29999999998}
2021-02-19 12:53:32.363837, Episode 2040, cumulative total regret: {'ucrl2_mc': 17025.40000000001, 'ucrl2': 49911.89999999997}
2021-02-19 12:54:09.673832, Episode 2050, cumulative total regret: {'ucrl2_mc': 17030.300000000007, 'ucrl2': 49952.59

2021-02-19 13:30:44.644119, Episode 2640, cumulative total regret: {'ucrl2_mc': 18386.699999999993, 'ucrl2': 53588.50000000002}
2021-02-19 13:31:21.958250, Episode 2650, cumulative total regret: {'ucrl2_mc': 18430.0, 'ucrl2': 53635.200000000026}
2021-02-19 13:31:59.939007, Episode 2660, cumulative total regret: {'ucrl2_mc': 18489.899999999994, 'ucrl2': 53723.60000000003}
2021-02-19 13:32:37.539484, Episode 2670, cumulative total regret: {'ucrl2_mc': 18507.299999999996, 'ucrl2': 53758.00000000004}
2021-02-19 13:33:14.858299, Episode 2680, cumulative total regret: {'ucrl2_mc': 18526.499999999996, 'ucrl2': 53811.80000000004}
2021-02-19 13:33:52.469225, Episode 2690, cumulative total regret: {'ucrl2_mc': 18546.899999999998, 'ucrl2': 53854.40000000003}
2021-02-19 13:34:30.193008, Episode 2700, cumulative total regret: {'ucrl2_mc': 18567.899999999994, 'ucrl2': 53892.60000000004}
2021-02-19 13:35:07.065043, Episode 2710, cumulative total regret: {'ucrl2_mc': 18568.59999999999, 'ucrl2': 53965.

2021-02-19 14:11:16.076788, Episode 3290, cumulative total regret: {'ucrl2_mc': 20017.60000000001, 'ucrl2': 56148.50000000004}
2021-02-19 14:11:53.907663, Episode 3300, cumulative total regret: {'ucrl2_mc': 20030.00000000001, 'ucrl2': 56231.20000000004}
2021-02-19 14:12:31.434620, Episode 3310, cumulative total regret: {'ucrl2_mc': 20043.400000000012, 'ucrl2': 56275.50000000005}
2021-02-19 14:13:08.719844, Episode 3320, cumulative total regret: {'ucrl2_mc': 20096.500000000015, 'ucrl2': 56303.10000000006}
2021-02-19 14:13:46.206136, Episode 3330, cumulative total regret: {'ucrl2_mc': 20167.500000000015, 'ucrl2': 56355.200000000055}
2021-02-19 14:14:23.658998, Episode 3340, cumulative total regret: {'ucrl2_mc': 20203.000000000015, 'ucrl2': 56366.20000000006}
2021-02-19 14:15:01.331515, Episode 3350, cumulative total regret: {'ucrl2_mc': 20213.400000000012, 'ucrl2': 56417.40000000007}
2021-02-19 14:15:39.716051, Episode 3360, cumulative total regret: {'ucrl2_mc': 20265.50000000001, 'ucrl2

2021-02-19 14:58:33.628117, Episode 3940, cumulative total regret: {'ucrl2_mc': 21544.099999999984, 'ucrl2': 57684.40000000009}
2021-02-19 14:59:19.991210, Episode 3950, cumulative total regret: {'ucrl2_mc': 21578.99999999998, 'ucrl2': 57717.900000000074}
2021-02-19 15:00:06.901493, Episode 3960, cumulative total regret: {'ucrl2_mc': 21637.69999999999, 'ucrl2': 57743.30000000008}
2021-02-19 15:00:51.968600, Episode 3970, cumulative total regret: {'ucrl2_mc': 21641.399999999987, 'ucrl2': 57779.90000000009}
2021-02-19 15:01:38.337625, Episode 3980, cumulative total regret: {'ucrl2_mc': 21658.09999999998, 'ucrl2': 57809.60000000008}
2021-02-19 15:02:23.361819, Episode 3990, cumulative total regret: {'ucrl2_mc': 21711.899999999983, 'ucrl2': 57855.20000000008}
2021-02-19 15:03:08.750466, Episode 4000, cumulative total regret: {'ucrl2_mc': 21737.399999999987, 'ucrl2': 57902.00000000007}
2021-02-19 15:03:51.994283, Episode 4010, cumulative total regret: {'ucrl2_mc': 21778.999999999985, 'ucrl2

2021-02-19 15:43:52.608079, Episode 4590, cumulative total regret: {'ucrl2_mc': 22971.899999999976, 'ucrl2': 59157.80000000006}
2021-02-19 15:44:30.157568, Episode 4600, cumulative total regret: {'ucrl2_mc': 22952.699999999975, 'ucrl2': 59157.100000000064}
2021-02-19 15:45:07.513003, Episode 4610, cumulative total regret: {'ucrl2_mc': 22964.199999999975, 'ucrl2': 59185.000000000065}
2021-02-19 15:45:43.331742, Episode 4620, cumulative total regret: {'ucrl2_mc': 22991.799999999977, 'ucrl2': 59231.40000000006}
2021-02-19 15:46:20.281647, Episode 4630, cumulative total regret: {'ucrl2_mc': 23055.69999999998, 'ucrl2': 59241.90000000006}
2021-02-19 15:46:56.630657, Episode 4640, cumulative total regret: {'ucrl2_mc': 23082.199999999975, 'ucrl2': 59238.700000000055}
2021-02-19 15:47:34.150446, Episode 4650, cumulative total regret: {'ucrl2_mc': 23094.199999999975, 'ucrl2': 59253.20000000006}
2021-02-19 15:48:10.060448, Episode 4660, cumulative total regret: {'ucrl2_mc': 23157.799999999974, 'u

In [57]:
''' plot the regret results for multiple teams (Figure 5) '''

# you can run the above cell or load the saved regret results 

# for figure 4(b) load file `total_regret_0`, for figure 4(a) load file `total_regret_01`
with open('outputs/agents/multiple_teams/total_regret_1', 'rb') as file:
    regrets = pickle.load(file)

width = 1.3
height = 1

latexify(fig_height=height, fig_width=width, font_size=7, legend_size=5)
line_width = 0.8
line_style='solid'

plt.figure(1)
fig, ax = plt.subplots()
ax = format_axes(ax)
ax.ticklabel_format(style='sci', scilimits=(3, 3), axis='y')
ax.ticklabel_format(style='sci', scilimits=(3, 3), axis='x')
ax.set_ylabel(r'Regret, $R(T)$')
ax.set_xlabel(r'Episode, $k$')

ax.plot(np.cumsum(regrets['ucrl2_mc']),color=COLORS[0], label=r'UCRL2-MC', linestyle=line_style,
        linewidth=line_width)
ax.plot(np.cumsum(regrets['ucrl2']), color=COLORS[1], label=r'UCRL2', linestyle=line_style,
        linewidth=line_width)

ax.legend(frameon=False, loc=(0.0, 0.76))
plt.subplots_adjust(left=0.23, bottom=0.28, right=0.98, top=0.87, wspace=0, hspace=0)
# plt.show()
fig.savefig('outputs/multiple_teams_s1.pdf')
plt.close()